In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install numpy==1.26.4 --quiet
import os
os._exit(0)  # Restart the runtime to apply downgrade immediately

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 106.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# pip install -q transformers datasets scikit-learn pandas

In [2]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
import evaluate
import torch

In [4]:
# Dataset
train_path = "/content/drive/MyDrive/ClinicalNotesGen/Data/speaker_role/train.parquet"
val_path = "/content/drive/MyDrive/ClinicalNotesGen/Data/speaker_role/val.parquet"

# Model output
output_dir = "/content/drive/MyDrive/ClinicalNotesGen/RoleClassifier/Train_3/Checkpoints"
log_dir = "/content/drive/MyDrive/ClinicalNotesGen/RoleClassifier/Train_3/Logs"
save_dir = "/content/drive/MyDrive/ClinicalNotesGen/RoleClassifier/Train_3/Final_Checkpoints"

In [5]:
# Step 1: Load dataset
train_df = pd.read_parquet(train_path)
val_df = pd.read_parquet(val_path)
print(train_df.shape)
print(val_df.shape)

(14562, 6)
(1619, 6)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
train_df.head(3)

,dialogue_index,speaker_role,text,utterance_count,dialogue_length,turn_indices
0,7944,patient,"No, this is my first time. Yes, that's right. ...",8,16,"[1, 3, 5, 7, 9, 11, 13, 15]"
1,8398,patient,"Hi doc, I've been having these issues for arou...",7,14,"[1, 3, 5, 7, 9, 11, 13]"
2,2204,patient,"Yeah, it was really scary. I suddenly felt my ...",6,12,"[1, 3, 5, 7, 9, 11]"


In [8]:
# Encode labels
label2id = {"doctor": 0, "patient": 1}
id2label = {0: "doctor", 1: "patient"}
train_df["label"] = train_df["speaker_role"].map(label2id)
val_df["label"] = val_df["speaker_role"].map(label2id)

In [9]:
# Convert to Hugging Face Datasets
dataset_train = Dataset.from_pandas(train_df.reset_index(drop=True))
dataset_val = Dataset.from_pandas(val_df.reset_index(drop=True))

In [10]:
dataset_train

Dataset({
    features: ['dialogue_index', 'speaker_role', 'text', 'utterance_count', 'dialogue_length', 'turn_indices', 'label'],
    num_rows: 14562
})

In [11]:
dataset_val

Dataset({
    features: ['dialogue_index', 'speaker_role', 'text', 'utterance_count', 'dialogue_length', 'turn_indices', 'label'],
    num_rows: 1619
})

In [12]:
# Step 3: Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
MAX_LENGTH = 260

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

dataset_train = dataset_train.map(tokenize, batched=True)
dataset_val = dataset_val.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/14562 [00:00<?, ? examples/s]

Map:   0%|          | 0/1619 [00:00<?, ? examples/s]

In [13]:
# Set format for PyTorch
columns = ['input_ids', 'attention_mask', 'label']
dataset_train.set_format(type='torch', columns=columns)
dataset_val.set_format(type='torch', columns=columns)

In [14]:
# Step 4: Load model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Step 5: Training setup

args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="steps",
    eval_steps=20,

    save_strategy="steps",
    save_steps=20,
    save_total_limit=2,

    # Logs
    logging_strategy="steps",
    logging_steps=20,
    logging_dir=log_dir,

    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    learning_rate=5e-5,
    warmup_steps=500,
    lr_scheduler_type="cosine",
    weight_decay=0.01,

    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

# Step 6: Metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }

# Step 7: Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/tmp/ipython-input-15-2882923661.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
# Step 8: Train
trainer.train()

# Step 9: Save final model
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"✅ Training complete. Model saved to {save_dir}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: montserrat400italic (montserrat400italic-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
20,0.645600,0.498565,0.949969,0.949935
40,0.432000,0.334550,0.969734,0.969690
60,0.287900,0.214249,0.992588,0.992587
80,0.178200,0.120027,0.997529,0.997529
100,0.092200,0.048102,0.999382,0.999382
120,0.032800,0.013561,0.999382,0.999382
140,0.013200,0.007262,0.999382,0.999382
160,0.004700,0.004556,0.999382,0.999382
180,0.002900,0.004219,0.999382,0.999382


✅ Training complete. Model saved to /content/drive/MyDrive/ClinicalNotesGen/RoleClassifier/Train_3/Final_Checkpoints
